In [31]:
import json
import numpy as np
import pandas as pd
from inline_sql import sql
import matplotlib.pyplot as plt

In [32]:
pozos = pd.read_csv("capitulo-iv-pozos.csv")
produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")

C:\Users\carri\AppData\Local\Temp\ipykernel_8008\1836493753.py:2: DtypeWarning: Columns (16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")


In [33]:
pozos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84332 entries, 0 to 84331
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    84332 non-null  object 
 1   idpozo                   84332 non-null  int64  
 2   area                     84332 non-null  object 
 3   cod_area                 84332 non-null  object 
 4   empresa                  83435 non-null  object 
 5   yacimiento               84332 non-null  object 
 6   cod_yacimiento           84332 non-null  object 
 7   formacion                81517 non-null  object 
 8   cuenca                   84332 non-null  object 
 9   provincia                84332 non-null  object 
 10  cota                     84332 non-null  float64
 11  profundidad              84332 non-null  float64
 12  clasificacion            84332 non-null  object 
 13  subclasificacion         84332 non-null  object 
 14  tipo_recurso          

In [34]:
produccion['index'] = produccion.index
produccion_nonula = sql^ """
                         SELECT *
                         FROM produccion AS p1
                         WHERE EXISTS ( SELECT p2.idpozo, SUM(p2.prod_pet) AS prod_pet_tot, SUM(p2.prod_gas) AS prod_gas_tot
                                        FROM produccion AS p2
                                        WHERE p1.idpozo = p2.idpozo
                                        GROUP BY p2.idpozo
                                        HAVING prod_pet_tot != 0 OR prod_gas_tot != 0
                                      )
                         """
produccion_nonula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274497 entries, 0 to 274496
Data columns (total 39 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idempresa               274497 non-null  object 
 1   anio                    274497 non-null  int64  
 2   mes                     274497 non-null  int64  
 3   idpozo                  274497 non-null  int64  
 4   prod_pet                274497 non-null  float64
 5   prod_gas                274497 non-null  float64
 6   prod_agua               274497 non-null  float64
 7   iny_agua                274497 non-null  float64
 8   iny_gas                 274497 non-null  float64
 9   iny_co2                 274497 non-null  float64
 10  iny_otro                274497 non-null  float64
 11  tef                     274497 non-null  float64
 12  vida_util               90 non-null      float64
 13  tipoextraccion          274475 non-null  object 
 14  tipoestado          

In [35]:
pozos = pozos[pozos['idpozo'].isin(produccion_nonula['idpozo'].unique())]
pozos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27727 entries, 0 to 84330
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    27727 non-null  object 
 1   idpozo                   27727 non-null  int64  
 2   area                     27727 non-null  object 
 3   cod_area                 27727 non-null  object 
 4   empresa                  27727 non-null  object 
 5   yacimiento               27727 non-null  object 
 6   cod_yacimiento           27727 non-null  object 
 7   formacion                27699 non-null  object 
 8   cuenca                   27727 non-null  object 
 9   provincia                27727 non-null  object 
 10  cota                     27727 non-null  float64
 11  profundidad              27727 non-null  float64
 12  clasificacion            27727 non-null  object 
 13  subclasificacion         27727 non-null  object 
 14  tipo_recurso             27

In [36]:
pozos['geojson'].info()

<class 'pandas.core.series.Series'>
Index: 27727 entries, 0 to 84330
Series name: geojson
Non-Null Count  Dtype 
--------------  ----- 
27727 non-null  object
dtypes: object(1)
memory usage: 433.2+ KB


In [37]:
pozos['coords'] = pozos['geojson'].apply(json.loads)
pozos['coords'] = pozos['coords'].apply(lambda x: x['coordinates'])
pozos

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipopozo,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,coords
0,CH.CH.EaLE.x-1,212,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,Petrolífero,Bombeo Mecánico,Extracción Efectiva,1996-10-30,1996-11-13,1996-11-17,1996-12-07,"{""type"":""Point"",""coordinates"":[-68.28785299999...",0101000020E61000008D43FD2E6C1251C00E4B033FAACB...,"[-68.287853, -45.591133]"
2,CH.CH.EaLE-3,214,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,Petrolífero,Bombeo Mecánico,Extracción Efectiva,1997-01-30,1997-02-07,1997-03-02,1997-03-10,"{""type"":""Point"",""coordinates"":[-68.28387800000...",0101000020E61000007383A10E2B1251C082548A1D8DCB...,"[-68.283878, -45.590244]"
3,CH.CH.EaLE-4,215,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,Petrolífero,Bombeo Mecánico,Extracción Efectiva,1997-01-13,1997-01-20,1997-01-23,1997-02-07,"{""type"":""Point"",""coordinates"":[-68.28948300000...",0101000020E6100000DF6FB4E3861251C0A148F7730ACC...,"[-68.289483, -45.594069]"
4,CH.CH.EaLE-5,216,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,Petrolífero,Bombeo Mecánico,Extracción Efectiva,1997-02-16,1997-02-24,1997-03-10,1997-03-18,"{""type"":""Point"",""coordinates"":[-68.29447500000...",0101000020E6100000569FABADD81251C0B98D06F016CC...,"[-68.294475, -45.59445]"
5,CH.CH.EaLE-6,217,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,Petrolífero,Bombeo Mecánico,Extracción Efectiva,1997-01-22,1997-01-30,1997-02-10,1997-02-27,"{""type"":""Point"",""coordinates"":[-68.29731700000...",0101000020E6100000DFE2E13D071351C006668522DDCB...,"[-68.297317, -45.592686]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84326,SHE.Nq.SB-1045(h),165739,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,Neuquén,...,Petrolífero,Surgencia Natural,Extracción Efectiva,2022-07-24,2023-07-07,2024-04-16,2024-10-01,"{""type"":""Point"",""coordinates"":[-68.43798545575...",0101000020E6100000EE2526F4071C51C05F938CD2623C...,"[-68.4379854557553, -38.4717658220677]"
84327,SHE.Nq.SB-1046(h),165740,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,Neuquén,...,Petrolífero,Surgencia Natural,Extracción Efectiva,2022-08-10,2023-07-26,2024-04-16,2024-08-13,"{""type"":""Point"",""coordinates"":[-68.43781272688...",0101000020E610000020EFAB1F051C51C0CEA160CE623C...,"[-68.4378127268833, -38.4717653247677]"
84328,SHE.Nq.SB-1047(h),165741,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,Neuquén,...,Petrolífero,Surgencia Natural,Extracción Efectiva,2022-08-23,2023-08-18,2024-04-16,2024-08-14,"{""type"":""Point"",""coordinates"":[-68.43764185701...",0101000020E61000002AD0FD52021C51C0E9B258C7623C...,"[-68.4376418570179, -38.4717644866085]"
84329,SHE.Nq.SB-1048(h),165742,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,Neuquén,...,Petrolífero,Surgencia Natural,Extracción Efectiva,2022-09-05,2023-09-08,2024-04-16,2024-08-12,"{""type"":""Point"",""coordinates"":[-68.43746999801...",0101000020E6100000E99C2982FF1B51C0195847C4623C...,"[-68.4374699980141, -38.4717641208992]"


In [19]:
pozos = pd.concat([pozos, coordenadas], axis=1)
pozos

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat,lon,lat
0,CH.CH.EaLE.x-1,212.0,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,1996-10-30,1996-11-13,1996-11-17,1996-12-07,"{""type"":""Point"",""coordinates"":[-68.28785299999...",0101000020E61000008D43FD2E6C1251C00E4B033FAACB...,-68.287853,-45.591133,-68.287853,-45.591133
2,CH.CH.EaLE-3,214.0,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,1997-01-30,1997-02-07,1997-03-02,1997-03-10,"{""type"":""Point"",""coordinates"":[-68.28387800000...",0101000020E61000007383A10E2B1251C082548A1D8DCB...,-68.289483,-45.594069,-68.289483,-45.594069
3,CH.CH.EaLE-4,215.0,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,1997-01-13,1997-01-20,1997-01-23,1997-02-07,"{""type"":""Point"",""coordinates"":[-68.28948300000...",0101000020E6100000DF6FB4E3861251C0A148F7730ACC...,-68.294475,-45.594450,-68.294475,-45.594450
4,CH.CH.EaLE-5,216.0,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,1997-02-16,1997-02-24,1997-03-10,1997-03-18,"{""type"":""Point"",""coordinates"":[-68.29447500000...",0101000020E6100000569FABADD81251C0B98D06F016CC...,-68.297317,-45.592686,-68.297317,-45.592686
5,CH.CH.EaLE-6,217.0,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,Chubut,...,1997-01-22,1997-01-30,1997-02-10,1997-02-27,"{""type"":""Point"",""coordinates"":[-68.29731700000...",0101000020E6100000DFE2E13D071351C006668522DDCB...,-68.295372,-45.590272,-68.295372,-45.590272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-68.437985,-38.471766,-68.437985,-38.471766
27723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-68.437813,-38.471765,-68.437813,-38.471765
27724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-68.437642,-38.471764,-68.437642,-38.471764
27725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-68.437470,-38.471764,-68.437470,-38.471764


# Unicidad de coordenadas

In [8]:
geom_dup = pozos[pozos['geom'].duplicated(keep=False)]
geom_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24021 entries, 239 to 27726
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    2387 non-null   object 
 1   idpozo                   2387 non-null   float64
 2   area                     2387 non-null   object 
 3   cod_area                 2387 non-null   object 
 4   empresa                  2387 non-null   object 
 5   yacimiento               2387 non-null   object 
 6   cod_yacimiento           2387 non-null   object 
 7   formacion                2387 non-null   object 
 8   cuenca                   2387 non-null   object 
 9   provincia                2387 non-null   object 
 10  cota                     2387 non-null   float64
 11  profundidad              2387 non-null   float64
 12  clasificacion            2387 non-null   object 
 13  subclasificacion         2387 non-null   object 
 14  tipo_recurso             

In [9]:
geojson_dup = pozos[pozos['geojson'].duplicated(keep=False)]
geojson_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24021 entries, 239 to 27726
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    2387 non-null   object 
 1   idpozo                   2387 non-null   float64
 2   area                     2387 non-null   object 
 3   cod_area                 2387 non-null   object 
 4   empresa                  2387 non-null   object 
 5   yacimiento               2387 non-null   object 
 6   cod_yacimiento           2387 non-null   object 
 7   formacion                2387 non-null   object 
 8   cuenca                   2387 non-null   object 
 9   provincia                2387 non-null   object 
 10  cota                     2387 non-null   float64
 11  profundidad              2387 non-null   float64
 12  clasificacion            2387 non-null   object 
 13  subclasificacion         2387 non-null   object 
 14  tipo_recurso             

In [10]:
same_coords = geom_dup.index == geojson_dup.index
False in same_coords

False

In [11]:
len(geom_dup['geom'].unique())

1093

In [12]:
geom_dup[(geom_dup['geom'] == geom_dup['geom'].iloc[500])]

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat
42188,YPF.Nq.EL-22,109903.0,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,rayoso,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2004-07-02,2004-07-15,2004-07-29,2004-08-14,"{""type"":""Point"",""coordinates"":[-69.2757194,-37...",0101000020E6100000DB77FB62A55151C0F5A9190838AE...,NaN,NaN
42189,YPF.Nq.EL-22,109904.0,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2004-07-02,2004-07-15,2004-07-29,2004-08-14,"{""type"":""Point"",""coordinates"":[-69.2757194,-37...",0101000020E6100000DB77FB62A55151C0F5A9190838AE...,NaN,NaN


In [13]:
sample_dup = produccion[(produccion['idpozo'] == 109903) | (produccion['idpozo'] == 109904)]
sample_dup[(sample_dup['prod_pet'] != 0) & (sample_dup['prod_gas'] != 0)]

,idempresa,anio,mes,idpozo,prod_pet,prod_gas,prod_agua,iny_agua,iny_gas,iny_co2,...,areayacimiento,cuenca,provincia,tipo_de_recurso,proyecto,clasificacion,subclasificacion,sub_tipo_recurso,fecha_data,index
36661,YPF,2024,1,109904,15.78,0.07,161.95,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36661
36662,YPF,2024,1,109903,36.83,0.15,377.89,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36662
75000,YPF,2024,2,109904,13.38,0.06,157.97,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75000
75001,YPF,2024,2,109903,31.22,0.15,368.58,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75001
112625,YPF,2024,3,109904,15.13,0.07,172.38,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112625
112626,YPF,2024,3,109903,35.30,0.16,402.23,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112626
154639,YPF,2024,4,109904,15.16,0.07,174.34,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154639
154640,YPF,2024,4,109903,35.38,0.16,406.80,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154640
190633,YPF,2024,5,109904,12.93,0.06,153.19,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190633
190634,YPF,2024,5,109903,30.18,0.14,357.43,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190634


In [14]:
geom_dup.loc[42188] == geom_dup.loc[42189]

sigla                       True
idpozo                     False
area                        True
cod_area                    True
empresa                     True
yacimiento                  True
cod_yacimiento              True
formacion                  False
cuenca                      True
provincia                   True
cota                        True
profundidad                 True
clasificacion               True
subclasificacion            True
tipo_recurso                True
sub_tipo_recurso            True
gasplus                     True
tipopozo                    True
tipoextraccion              True
tipoestado                  True
adjiv_fecha_inicio_perf     True
adjiv_fecha_fin_perf        True
adjiv_fecha_inicio_term     True
adjiv_fecha_fin_term        True
geojson                     True
geom                        True
lon                        False
lat                        False
dtype: bool

# Unicidad de sigla

In [15]:
sigla_dup = pozos[(~pozos['geom'].duplicated(keep=False))]
sigla_dup = sigla_dup[(sigla_dup['sigla'].duplicated(keep=False))]
sigla_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 224 to 84301
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    294 non-null    object 
 1   idpozo                   294 non-null    float64
 2   area                     294 non-null    object 
 3   cod_area                 294 non-null    object 
 4   empresa                  294 non-null    object 
 5   yacimiento               294 non-null    object 
 6   cod_yacimiento           294 non-null    object 
 7   formacion                294 non-null    object 
 8   cuenca                   294 non-null    object 
 9   provincia                294 non-null    object 
 10  cota                     294 non-null    float64
 11  profundidad              294 non-null    float64
 12  clasificacion            294 non-null    object 
 13  subclasificacion         294 non-null    object 
 14  tipo_recurso             29

In [16]:
dup = sigla_dup[sigla_dup['sigla'] == sigla_dup['sigla'].iloc[123]]
dup

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat
60268,YPF.Md.NDB-226,129599.0,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,DESFILADERO BAYO,DBA,agrio,NEUQUINA,Mendoza,...,Bombeo Mecánico,Extracción Efectiva,2008-01-21,2008-02-05,2008-02-20,2008-03-13,"{""type"":""Point"",""coordinates"":[-69.03750091230...",0101000020E610000084F9396A664251C09CC2E46B0AA9...,NaN,NaN
60269,YPF.Md.NDB-226,129600.0,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,DESFILADERO BAYO,DBA,mulichinco,NEUQUINA,Mendoza,...,Bombeo Mecánico,Extracción Efectiva,2008-01-21,2008-02-05,2008-02-20,2008-03-13,"{""type"":""Point"",""coordinates"":[-69.03750091210...",0101000020E61000008AC2396A664251C0B5E6E36B0AA9...,NaN,NaN
69029,YPF.Md.NDB-226,144637.0,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,DESFILADERO BAYO,DBA,rayoso,NEUQUINA,Mendoza,...,Bombeo Mecánico,Extracción Efectiva,2008-01-21,2008-02-05,2008-02-20,2008-03-13,"{""type"":""Point"",""coordinates"":[-69.03749999999...",0101000020E610000066666666664251C07E8CB96B09A9...,NaN,NaN


In [17]:
np.isclose(dup['lon'], dup['lon'].iloc[0])

array([False, False, False])

In [18]:
dup['lon']

60268   NaN
60269   NaN
69029   NaN
Name: lon, dtype: float64